# Train a Simple Regression Model

The process of training a machine learning (ML) model can be thought of as fitting a
highly parameterized function to map inputs to outputs. An ML algorithm needs to train
numerous examples of input and output pairs to accurately map an input to an output,
i. e., make a prediction. After training, result is referred to a trained ML model or an artifact.

This tutorial will detail how we can use AMPL tools to train a regression model to predict 
how much a compound will inhibit the kcna3 gene as measured by pIC50. 
We will train a random forest model using the following inputs:

1. The curated kcna3 dataset from tutorial 2.
2. The split file generated in tutorial 3.
3. RDKit features calculated by the AMPL pipeline.

We will explain the use of descriptors, how to evaulate model performance,
and where the model is saved as a .tar.gz file.

> **Note** Training a random forest model and splitting the dataset are non-deterministic. 
You will obtain a slightly different random forest model by running this tutorial each time.

## Model Training (using already split data)

We will use the curated dataset created in tutorial 2 and the split file 
created in tutorial 3 to build a json file for training. We set `"previously_split": "True"`
 and set the `split_uuid`. 
Here, we will use `"split_uuid" : "c0313c63-8936-4297-925b-ee537b66dd89"`; 
the uuid for the scaffold split created in tutorial 3.

AMPL provides an extensive featurization module that can generate a 
variety of molecular feature types, given SMILES strings as input. 
For demonstration purposes, we choose to use RDKit features in this tutorial.

When the featurized dataset is not previously saved for curated_kcna3_ic50, 
AMPL will create a featurized dataset and save it in a folder called `scaled_descriptors` 
as a csv file e.g. `dataset/scaled_descriptors/curated_kcna3_ic50_with_rdkit_raw_descriptors.csv`

In [8]:
# importing relevant libraries
import pandas as pd
from atomsci.ddm.pipeline import model_pipeline as mp
from atomsci.ddm.pipeline import parameter_parser as parse

# Set up
dataset_file = 'dataset/curated_kcna3_ic50.csv'
odir='dataset'

response_col = "avg_pIC50"
compound_id = "compound_id"
smiles_col = "base_rdkit_smiles"

params = {
        "verbose": "True",
        "system": "LC",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "regression",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        "previously_split": "True",
        "split_uuid" : "c0313c63-8936-4297-925b-ee537b66dd89",
        "split_only": "False",
        "featurizer": "computed_descriptors",
        "descriptor_type" : "rdkit_raw",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        "rerun": "False",
        "result_dir": odir
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

INFO:ATOM:Created a dataset hash '389b161b7a4eb2304323a7dfddacfacc' from dataset_key '/usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/examples/tutorials2023/dataset/curated_kcna3_ic50.csv'
DEBUG:ATOM:Attempting to load featurized dataset
DEBUG:ATOM:Got dataset, attempting to extract data
DEBUG:ATOM:Creating deepchem dataset
INFO:ATOM:Using prefeaturized data; number of features = 200
/usr/WS1/he6/AMPL_virtualenv_1.6/lib/python3.9/site-packages/deepchem/data/datasets.py:473: RuntimeWarning: overflow encountered in multiply
  X_m2 += dx * (X - X_means)
INFO:ATOM:Wrote transformers to dataset/curated_kcna3_ic50/RF_computed_descriptors_scaffold_regression/a6c93e7b-978b-4b2e-9358-70ff64fedadd/transformers.pkl
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
/usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/pipeline/transformations.py:255: RuntimeWarning: invalid value encountered in divide
  X = np.nan_to_num((X - self.X_means) * X_weight / self.X_stds)
INFO:ATOM:Transforming re

## Model Training (Split data and train)

It is possible to split and train a model in one step. 
Here, we set `"previously_split": "False"` and not have a split_uuid parameter. 
AMPL splits the data by the type of split specified in the splitter parameter, 
scaffold in this example, and writes the split file in
`dataset/curated_kcna3_ic50_train_valid_test_scaffold_{split_uuid}.csv.` 
After training, AMPL saves the model and all of its parameters as a tarball in the result_dir.

In [9]:
response_col = "avg_pIC50"
compound_id = "compound_id"
smiles_col = "base_rdkit_smiles"

params = {
        "verbose": "True",
        "system": "LC",
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "regression",
        "dataset_key": dataset_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        "previously_split": "False",
        "split_only": "False",
        "splitter": "scaffold",
        "split_valid_frac": "0.15",
        "split_test_frac": "0.15",
        "featurizer": "computed_descriptors",
        "descriptor_type" : "rdkit_raw",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        "rerun": "False",
        "result_dir": odir
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

INFO:ATOM:Created a dataset hash '389b161b7a4eb2304323a7dfddacfacc' from dataset_key '/usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/examples/tutorials2023/dataset/curated_kcna3_ic50.csv'
DEBUG:ATOM:Attempting to load featurized dataset
DEBUG:ATOM:Got dataset, attempting to extract data
DEBUG:ATOM:Creating deepchem dataset
INFO:ATOM:Using prefeaturized data; number of features = 200
/usr/WS1/he6/AMPL_virtualenv_1.6/lib/python3.9/site-packages/deepchem/data/datasets.py:473: RuntimeWarning: overflow encountered in multiply
  X_m2 += dx * (X - X_means)
INFO:ATOM:Wrote transformers to dataset/curated_kcna3_ic50/RF_computed_descriptors_scaffold_regression/038baafa-e754-4110-b3aa-06c96d431f80/transformers.pkl
INFO:ATOM:Transforming response data
INFO:ATOM:Transforming feature data
/usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/pipeline/transformations.py:255: RuntimeWarning: invalid value encountered in divide
  X = np.nan_to_num((X - self.X_means) * X_weight / self.X_stds)
INFO:ATOM:Transforming re

## Performance of the Model
We evalualte model performance by measuring how a accurate 
models are on validation and test sets. 
The validation set is used while optimizing the model and choosing the best
parameter settings. Then the performance on the test set is to final judge of
model performance.

AMPL has several popular metrics to evaulate regression models; 
Mean Absolute Error (MAE), Mean Squared Error (MSE), Root Mean Squared Error (RMSE) and R² (R-Squared).
In our tutorials, we will use R² metric to compare our models. The best model will have the highest
R² score.

> **Note** The model tracker client will not be supported in your environment.

In [10]:
# Model Performance
from atomsci.ddm.pipeline import compare_models as cm
pred_df = cm.get_filesystem_perf_results(odir, pred_type='regression')

Found data for 2 models under dataset


The pred_df dataframe has details about the model_uuid, model_path, ampl_version, model_type, features, splitter and the results for popular metrics that help evaluate the performance. Let us view the contents of the pred_df dataframe.

In [11]:
pred_df.to_csv('./dataset/pred_df.csv')

In [12]:
# View the pred_df dataframe
pred_df

,model_uuid,model_path,ampl_version,model_type,dataset_key,features,splitter,model_score_type,feature_transform_type,model_choice_score,...,rf_max_depth,max_epochs,best_epoch,learning_rate,layer_sizes,dropouts,xgb_gamma,xgb_learning_rate,model_parameters_dict,feat_parameters_dict
0,a6c93e7b-978b-4b2e-9358-70ff64fedadd,dataset/curated_kcna3_ic50_model_a6c93e7b-978b...,1.6.0,RF,/usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/exa...,rdkit_raw,scaffold,r2,normalization,0.719773,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""rf_estimators"": 500, ""rf_max_depth"": null, ""...",{}
1,038baafa-e754-4110-b3aa-06c96d431f80,dataset/curated_kcna3_ic50_model_038baafa-e754...,1.6.0,RF,/usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/exa...,rdkit_raw,scaffold,r2,normalization,0.716345,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""rf_estimators"": 500, ""rf_max_depth"": null, ""...",{}


## Top Performing Model
To pick the top performing model, we sort the `best_valid_r2_score` column in descending order and pick the one that is maximum.

In [13]:
# Top performing model
top_model=pred_df.sort_values(by="best_valid_r2_score", ascending=False).iloc[0,:]
top_model

model_uuid                               a6c93e7b-978b-4b2e-9358-70ff64fedadd
model_path                  dataset/curated_kcna3_ic50_model_a6c93e7b-978b...
ampl_version                                                            1.6.0
model_type                                                                 RF
dataset_key                 /usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/exa...
features                                                            rdkit_raw
splitter                                                             scaffold
model_score_type                                                           r2
feature_transform_type                                          normalization
model_choice_score                                                   0.719773
best_train_r2_score                                                  0.975317
best_train_rms_score                                                 0.286939
best_train_mae_score                                            

## Model Tarball 
The model_path or the location of the tarball where the top performing model is saved is in `top_model.model_path`.

In [14]:
# Top performing model path
top_model.model_path

'dataset/curated_kcna3_ic50_model_a6c93e7b-978b-4b2e-9358-70ff64fedadd.tar.gz'